In [ ]:
## Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shap
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier, MLPRegressor
from keras.models import Sequential, Model
from keras.layers import Dense,Activation,Dropout, Flatten, Concatenate, concatenate, Input, Lambda
# from keras.layers.embeddings import Embedding
from keras.layers.normalization import batch_normalization 
from keras.utils import np_utils
from tqdm import tqdm
import ipywidgets as widgets
## Util Functions
# def convert_to_int()

def get_reshape(row):
    # print(row)
    return np.fromstring(row['grid'].replace('\n','')
                                    .replace('[','')
                                    .replace(']','')
                                    .replace('  ',' '), sep=' ').reshape(row['grid_rows'],row['grid_columns'])

def convert_obs(df):
    return df['observation'].apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))

def gen_heatmap(df,f, ax):
    df = df.reset_index()
    x_array = df['x'].to_numpy()
    y_array =df['y'].to_numpy()
    grid = np.zeros((df['grid_rows'][0],df['grid_columns'][0]))
    for i, j in zip(y_array,x_array):
        grid[i,j]+=1
    heat =sns.heatmap(grid,cmap='viridis',alpha =0.6,zorder=2,ax=ax)
    my_image = mpimg.imread(f)
    # plt.imshow(my_image)
    plt.title(df['experiment'][0])
    ax.heat.imshow(my_image,aspect=heat.get_aspect(),extent= heat.get_xlim() + heat.get_ylim(),zorder=1)
    # plt.close()
    # return heat

def keras_classifier(n_inputs,n_outputs):
    model = Sequential()
    model.add(Dense(1000, input_dim= n_inputs, activation='relu'))
    model.add(Dense(500, activation='relu'))
    model.add(Dense(300, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model


## Data Handling
### Reading in the data 
df1 = pd.read_csv('./data/data.csv')
df = pd.read_csv('./data/data_policy_switching.csv')
df.describe()
### Data Manipulation

# Modify the grid to be a correct numpy array representing the level
df1['grid'] = df1[['grid','grid_rows','grid_columns']].apply(lambda x: get_reshape(x),axis=1)
df['grid'] = df[['grid','grid_rows','grid_columns']].apply(lambda x: get_reshape(x),axis=1)

# format the observation space into numpy array
df1['observation'] = convert_obs(df1)
df['observation'] = convert_obs(df)

# Shuffle the data 
df1 = df1.sample(frac=1)
df = df.sample(frac=1)
df.head()
# split the observation into seperate columns
# observations = [dist_monster, dist_treaure, safe_dist_teasure, dist_potion, safe_dist_potion, dist_exit, safe_dist_exit, HP]
obs = ['dist_monster', 'dist_treaure', 'safe_dist_teasure', 'dist_potion', 'safe_dist_potion', 'dist_exit', 'safe_dist_exit', 'HP']

df1[obs] = pd.DataFrame(df1['observation'].to_list(), index= df1.index)
df[obs] = pd.DataFrame(df['observation'].to_list(), index= df.index)
df1.describe()

## Creating the dataset
data = df[['dist_monster', 'dist_treaure', 'safe_dist_teasure', 'dist_potion', 'safe_dist_potion', 'dist_exit', 'safe_dist_exit', 'HP', 'action']]
data
# data[obs] = (df[obs]-df[obs].mean())/df[obs].std()
y_data = pd.get_dummies(df['action'])
print(y_data)
sns.pairplot(data=data, hue = 'action')
x_data = data[obs].to_numpy()
y_data = data['action']
print(y_data)
### Split data
x_train, x_test,y_train, y_test = train_test_split(x_data,y_data,test_size=.2, random_state=42)
### create and train model (sklearn logistic regression)
model_LG = sklearn.linear_model.LogisticRegression(max_iter=100000000, tol=1e-12)
model_LG.fit(x_train,y_train)
model_LG.score(x_test,y_test)
# nn = MLPClassifier(solver="adam",alpha=1e-0001,hidden_layer_sizes=(8,8,8),random_state=42, max_iter=10000000000000)
# nn.fit(data[obs].values,data['action'].values)
# nn.score(x_test,y_test)

nn = MLPClassifier(solver="adam",alpha=1e-0001,hidden_layer_sizes=(20,20,20),random_state=42, max_iter=10000000000000)
nn.fit(df[obs].values,df['action'].values)
nn.score(df[obs].values,df['action'].values)
nn1 = MLPRegressor(alpha=1e-1, hidden_layer_sizes=(8,8,8), random_state=0, max_iter=1000000000)
# nn1.fit(df[obs],y_train)
explainerKernel = shap.KernelExplainer(nn.predict_proba,data[obs])
shap_vals = explainerKernel.shap_values(x_test)
# shap.force_plot(explainer.expected_value[0],shap_vals[0],x_test)
# print the JS visualization code to the notebook
shap.initjs()
# shap.force_plot(explainer.expected_value[0],shap_vals[0],x_test)
shap.summary_plot(shap_vals,feature_names=data[obs].columns, class_names= y_data.columns)
# Create the list of all labels for the drop down list
list_of_labels = y_data.columns.to_list()

# Create a list of tuples so that the index of the label is what is returned
tuple_of_labels = list(zip(list_of_labels, range(len(list_of_labels))))

# Create a widget for the labels and then display the widget
current_label = widgets.Dropdown(options=tuple_of_labels,
                              value=0,
                              description='Select Label:'
                              )

# Display the dropdown list (Note: access index value with 'current_label.value')
current_label

shap.summary_plot(shap_values = shap_vals[current_label.value],feature_names= data[obs].columns)
# plot for all 4 classes
shap.bar_plot(shap_values = shap_vals[current_label.value])
explainer = shap.Explainer(nn.predict_proba, data[obs])
shap_values = explainer(data[obs])
shap.summary_plot(shap_values[current_label.value])


## Keras and big dataset
ldata =['dist_monster', 'dist_treaure', 'safe_dist_teasure', 'dist_potion', 'safe_dist_potion', 'dist_exit', 'safe_dist_exit', 'HP', 'action']
data = pd.concat([df[ldata],df1[ldata]], ignore_index=True)
data
x_data = data[obs]
y_data = pd.get_dummies(data['action'])
y_data.head()

nn = MLPClassifier(solver="adam",alpha=1e-0001,hidden_layer_sizes=(20,20,20),random_state=42, max_iter=10000000000000)
nn.fit(data[obs].values,data['action'].values)
nn.score(data[obs].values,data['action'].values)
model = keras_classifier(len(x_data.columns), len(y_data.columns))
model.summary()
model.fit(x_data.values,y_data.values,batch_size=20, epochs= 50, verbose=1)
def f(X):
    return model.predict([X[:,i] for i in range(X.shape[1])])
Kernel_explainer = shap.KernelExplainer(model.predict, x_data.iloc[:50,:])
# deep_explainer = shap.DeepExplainer(model,x_data.iloc[:50,:])
exact_explainer = shap.explainers.Exact(model.predict, x_data.iloc[:50,:])
kernel_shap_values = Kernel_explainer.shap_values(X = x_data.iloc[:50,:],nsamples=100)
shap.summary_plot(kernel_shap_values, features=x_data.iloc[:50,:])
kernel_shap_values
fig, axs = plt.subplots(2,3)
shap.summary_plot(kernel_shap_values, features=x_data.iloc[:50,:],)
